In [139]:
import numpy as np
import dill
import importlib
import os
import matplotlib.pyplot as plt
import data_visualization as dv
import copy
import numba as nb
import time
'''
v4 Uses the object imported from the py file class which is very similar to the Optimal_Stopping_Function_v3

As of 1/13/23, the only change is the addition of the decision to action delay
'''

'\nv4 Uses the object imported from the py file class which is very similar to the Optimal_Stopping_Function_v3\n\nAs of 1/13/23, the only change is the addition of the decision to action delay\n'

# Select Exp1 or Exp2


In [140]:
experiment = "Exp1"

In [141]:
import read_data_functions as rdf
import plot_functions as pf
import Optimal_Stopping_Object
importlib.reload(Optimal_Stopping_Object)
import Optimal_Stopping_Object as oso
import Subject_Object_v2

# Initial Thangs


In [142]:
plt.style.use("cashaback_dark")
wheel = dv.ColorWheel()

# Initial thangs
os.chdir(f"D:\OneDrive - University of Delaware - o365\Subject_Data\MatchPennies_Agent_{experiment}")
PATH = os.getcwd()
SAVE_PATH = f"D:\\OneDrive - University of Delaware - o365\\Subject_Data\\MatchPennies_Agent_{experiment}\\Figures\\"
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

# Fields pull and pull list
figures_pull_list = []
figures_pull_list_control = []
figures_pull_list_task = []
fields_pull = []
with open(PATH + "\\Figures_Pull_List.txt", "r") as pull_file:
    figures_pull_list = pull_file.read().splitlines()
with open(PATH + "\\Fields_Pull.txt", "r") as fields_pull:
    fields_pull = fields_pull.read().splitlines()
num_subjects = len(figures_pull_list)
if experiment == "Exp2":
    tp3_title = "Win = 1\nIncorrect = 0\nIndecision = 0"
    tp4_title = "Win = 1\nIncorrect = -1\nIndecision = 0"
    tp5_title = "Win = 1\nIncorrect = 0\nIndecision = -1"
    tp6_title = "Win = 1\nIncorrect = -1\nIndecision = -1"
    trial_block_titles = [tp3_title, tp4_title, tp5_title, tp6_title]
    num_blocks = len(trial_block_titles)
    xlabel = "Payoff Condition"
if experiment == "Exp1":
    tp3_title = "1000 (50)"
    tp4_title = "1000 (150)"
    tp5_title = "1100 (50)"
    tp6_title = "1100 (150)"
    tp7_title = "1200 (50)"
    tp8_title = "1200 (150)"
    trial_block_titles = [tp3_title, tp4_title, tp5_title, tp6_title, tp7_title, tp8_title]
    num_blocks = len(trial_block_titles)
    xlabel = "Mean [SD] Agent Decision Time (ms)"
num_subjects

20

# Pickle Data


In [143]:
if "subject_objects" not in locals():
    subject_objects = rdf.generate_subject_objects(experiment)
    subject_objects2 = copy.deepcopy(subject_objects)
    # group = Subject_Object_v2.Group(subject_objects, select_trials = 'All Trials', num_stds_for_reaction_time = 2,
    #                                     task_leave_time_metric_name = 'player_pos_task_leave_time', task_movement_time_metric_name = 'player_pos_task_movement_time',
    #                                     reaction_time_metric_name = 'player_pos_reaction_time', reaction_movement_time_metric_name = 'player_pos_reaction_movement_time')
    # group.analyze_data()
    group = Subject_Object_v2.Group(
        subject_objects2,
        select_trials="All Trials",
        num_stds_for_reaction_time=2,
        task_leave_time_metric_name="player_velocity_task_leave_time_thresh",
        task_movement_time_metric_name="player_velocity_task_movement_time_thresh",
        reaction_time_metric_name="player_velocity_reaction_time_thresh",
        reaction_movement_time_metric_name="player_velocity_reaction_movement_time_thresh",
    )
    group.analyze_data()

# Run fit sequence for each individual  

In [144]:
if experiment == "Exp1":
    rt = np.nanmedian(group.reaction_time, axis=1) - 15
    rt_sd = np.nanstd(group.reaction_time, axis=1)
elif experiment == "Exp2":
    rt    = np.nanmedian(group.react_reaction_time_only_react, axis=1) - 30
    rt_sd = np.nanmedian(np.nanstd(group.reaction_time, axis=2))

mt                   = np.min(np.nanmedian(group.player_task_movement_time, axis=2), axis=1)  # Get movement time for the condition where they tried the hardest
mt_sd                = np.median(np.nanstd(group.player_task_movement_time, axis=2), axis=1)
time_sd              = np.array([(np.nanstd(group.coincidence_reach_time, axis=1))] * num_blocks).T
perc_wins_both_reach = group.perc_gamble_wins_when_both_decide
gamble_sd            = np.nanstd(group.player_gamble_task_leave_time, axis=2)
agent_sds            = np.nanstd(group.agent_task_leave_time, axis=2)
agent_means          = np.nanmean(group.agent_task_leave_time, axis=2)

model_expected_list = {}
model_true_list = {}
for i in range(1):
    subname = figures_pull_list[i]
    # model_expected = oso.ModelConstructor(
    #     experiment=experiment,
    #     num_blocks=num_blocks,
    #     num_timesteps=1800,
    #     BETA_ON=False,
    #     agent_means=agent_means[i, :],
    #     agent_sds=agent_sds[i, :],
    #     reaction_time={"true": rt[i], "exp": rt[i]},
    #     movement_time={"true": mt[i], "exp": mt[i]},
    #     reaction_sd={"true": rt_sd[i], "exp": rt_sd[i]},
    #     movement_sd={"true": mt_sd[i], "exp": mt_sd[i]},
    #     timing_sd={"true": time_sd[i], "exp": time_sd[i]},
    #     perc_wins_when_both_reach=perc_wins_both_reach[i],
    #     gamble_delay_known=True,
    #     gamble_sd_known=True,
    #     gamble_decision_sd={"true": gamble_sd[i], "exp": 10},
    #     gamble_delay={"true": 125, "exp": 50},
    #     expected=True,
    # )

    model_true = oso.ModelConstructor(
        experiment=experiment,
        num_blocks=num_blocks,
        num_timesteps=1800,
        BETA_ON=False,
        agent_means=agent_means[i, :],
        agent_sds=agent_sds[i, :],
        reaction_time={"true": rt[i], "exp": rt[i]},
        movement_time={"true": mt[i], "exp": mt[i]},
        reaction_sd={"true": rt_sd[i], "exp": rt_sd[i]},
        movement_sd={"true": mt_sd[i], "exp": mt_sd[i]},
        timing_sd={"true": time_sd[i], "exp": time_sd[i]},
        perc_wins_when_both_reach=perc_wins_both_reach[i],
        gamble_delay_known=True,
        gamble_sd_known=True,
        gamble_decision_sd={"true": gamble_sd[i], "exp": 10},
        gamble_delay={"true": 125, "exp": 50},
        expected=False,
    )

    # Fit true model
    free_params_init = {'decision_time':np.array([900]*num_blocks),
                        'reaction_time':np.array([model_true.inputs.reaction_time['true']]*num_blocks),
                        }
    # get_true_metric = model_true.results.get_metric
    # init_decision_time = np.array([900]*num_blocks)
    # init_model_leave_time = get_true_metric(model_true.player_behavior.wtd_leave_target_time, metric_type="optimal")
    # init_model_wins       = get_true_metric(model_true.score_metrics.prob_win, metric_type="optimal")
    # init_model_incorrects       = get_true_metric(model_true.score_metrics.prob_incorrect, metric_type="optimal")
    # init_model_indecisions       = get_true_metric(model_true.score_metrics.prob_indecision, metric_type="optimal")

    # init_model_metrics = np.array([init_decision_time,init_model_leave_time,init_model_wins,init_model_incorrects,init_model_indecisions])

    targets = np.array(
        [np.nanmedian(group.player_task_leave_time, axis=2)[i], 
        group.player_perc_wins[i], 
        group.player_perc_incorrects[i], 
        group.player_perc_indecisions[i]]
    )
    metric_keys = ['wtd_leave_target_time','prob_win','prob_incorrect','prob_indecision']
    out = model_true.fit_multiple_parameters(
            free_params_init=free_params_init,
            targets=targets,
            metric_keys=metric_keys,
        )
    # # Fit expected model
    # model_expected.fit_model(
    #     model_expected.player_behavior.wtd_leave_target_time,
    #     np.nanmedian(group.player_task_leave_time, axis=2)[i],
    # )
    # Update dicts
    model_true_list.update({subname: model_true})
    # model_expected_list.update({subname: model_expected})

In [150]:

#TODO figure out this weirdness
model_true.results.fit_decision_time

array([882., 870., 968., 889., 960., 996.])

In [145]:
out

array([[882.64860689, 871.        , 968.44622054, 889.3911642 ,
        960.48615491, 996.47033484],
       [216.64658187, 203.18088497, 213.17701128, 213.53576043,
        192.69792708, 219.00677424]])